# 处理速度滑冰数据

In [250]:
import pandas as pd
import os

In [251]:
path = "/Volumes/ESD-USB/哈尔滨滑冰测试/速滑辅助鞋垫运动学数据.xlsx"
excelfile = pd.ExcelFile(path)
excelfile.sheet_names

['Sheet1',
 '直道滑行-不穿鞋垫',
 '直道滑行-穿鞋垫 ',
 '弯道滑行-不穿鞋垫',
 '弯道滑行-穿鞋垫',
 '特征时刻',
 '特征时刻_20231103',
 '特征时刻-20231031']

In [252]:
record_df = pd.read_excel(path, sheet_name="特征时刻")
record_df["文件名"] = record_df["日期"].astype(str) + "_" + \
            record_df["测试序号"].astype(str) + "_" + \
            record_df["姓名"].astype(str) + "_00" + \
            record_df["xsens号"].astype(str) + ".xlsx"

In [254]:
excel_dir = "/Volumes/ESD-USB/哈尔滨滑冰测试/excel"
for index, row in record_df.iterrows():
    date = row["日期"]
    test_number = row["测试序号"]
    name = row["姓名"]
    xsens_number = row["xsens号"]

    # 拼接文件名
    file_name = f"{date}_{test_number}_{name}-00{xsens_number}.xlsx"

    excel_path = os.path.join(excel_dir, file_name)
    
    #angle_df = pd.read_excel(excel_path, sheet_name="Joint Angles ZXY")
    #angular_vel_df = pd.read_excel(excel_path, sheet_name = "Segment Angular Velocity")
    

In [255]:
#读 excel 数据
excel_path = "/Volumes/ESD-USB/哈尔滨滑冰测试/excel/20231019_1_lininghao-001.xlsx"

angle_df = pd.read_excel(excel_path, sheet_name="Joint Angles ZXY")
angular_vel_df = pd.read_excel(excel_path, sheet_name = "Segment Angular Velocity")



In [256]:
seg_types = ["Left", "Right"]
axis = ["x", "y", "z"]
segments = [" Hip ", " Knee ", " Ankle "]
seg_orientations = ["Flexion/Extension","Abduction/Adduction", "Internal/External Rotation"]
seg_orientations_ankle = ["Dorsiflexion/Plantarflexion","Abduction/Adduction", "Internal/External Rotation"]

for seg_type in seg_types:
    angle_seg_types_colmuns = [seg_type + segment + seg_orientation
                 for seg_type in seg_types 
                 for segment in segments
                 for seg_orientation in ((seg_orientations_ankle if segment.strip()=="Ankle" else seg_orientations))]

In [257]:
# 构建列表头名称
def CreateSelectColumns(seg_type):
    select_seg_types_colmuns = [seg_type + segment + seg_orientation
                 for segment in segments
                 for seg_orientation in ((seg_orientations_ankle if segment.strip()=="Ankle" else seg_orientations))]
    return select_seg_types_colmuns

In [258]:
Left_select_columns = CreateSelectColumns("Left")
Right_select_columns = CreateSelectColumns("Right")

In [259]:
Left_contact_event = record_df.loc[1, "1直道触冰-左"]
Left_off_event = record_df.loc[1, "1直道离冰-左"]

# 左侧触冰、离冰时髋、膝、踝角度
Left_angle_contact = angle_df[Left_select_columns].loc[Left_contact_event]
Left_angle_off = angle_df[Left_select_columns].loc[Left_off_event]

frame = [Left_angle_contact, Left_angle_off]
Left_angles = pd.concat(frame, axis=0)


Right_contact_event = record_df.loc[1, "1直道触冰-右"]
Right_off_event = record_df.loc[1, "1直道离冰-右"]
# 右侧触冰、离冰时髋、膝、踝角度
Right_angle_contact = angle_df[Right_select_columns].loc[Right_contact_event]
Right_angle_off = angle_df[Right_select_columns].loc[Right_off_event]

frame = [Right_angle_contact, Right_angle_off]
Right_angles = pd.concat(frame, axis=0)

frame = [Left_angles, Right_angles]
result_frame_1 = pd.concat(frame, axis=0)


In [260]:
# 确定直道一次左脚触冰、离冰和右脚触冰离冰时角度
def GetEachTestResult(record_df, target_df, record_row, road_number, road_type):
    
    Left_contact_event = record_df.loc[record_row, f"{road_number}{road_type}触冰-左"]
    Left_off_event = record_df.loc[record_row, f"{road_number}{road_type}离冰-左"]
    
    Right_contact_event = record_df.loc[record_row, f"{road_number}{road_type}触冰-右"]
    Right_off_event = record_df.loc[record_row, f"{road_number}{road_type}离冰-右"]


    def GetEventAngles(target_df, select_columns, contact_event, off_event):
        # 左侧触冰、离冰时髋、膝、踝角度
        Left_angle_contact = target_df[select_columns].loc[contact_event]
        Left_angle_off = target_df[select_columns].loc[off_event]

        frame = [Left_angle_contact, Left_angle_off]
        angles = pd.concat(frame, axis=0)
        return angles


    Left_event_angles = GetEventAngles(target_df, Left_select_columns, Left_contact_event, Left_off_event)
    Right_event_angles = GetEventAngles(target_df, Right_select_columns, Right_contact_event, Right_off_event)

    frame = [Left_event_angles, Right_event_angles]
    Result_each_test = pd.concat(frame, axis=0)

    return Result_each_test

In [261]:
type(record_df.loc[record_row, ["姓名", "穿鞋", "xsens号"]])

pandas.core.series.Series

In [263]:
record_df.columns

Index(['日期', '测试序号', '姓名', '穿鞋', 'xsens号', '实际xsens号', '1直道开始', '1直道触冰-左',
       '1直道触冰-右', '1直道离冰-左', '1直道离冰-右', '1直道结束', '1弯道开始', '1弯道触冰-左', '1弯道触冰-右',
       '1弯道离冰-左', '1弯道离冰-右', '1弯道结束', '2直道开始', '2直道触冰-左', '2直道触冰-右', '2直道离冰-左',
       '2直道离冰-右', '2直道结束', '2弯道开始', '2弯道触冰-左', '2弯道触冰-右', '2弯道离冰-左', '2弯道离冰-右',
       '2弯道结束', '备注', '文件名'],
      dtype='object')

In [264]:
record_row = 8
road_type = "弯道"
traget_data = "角度"

# angle_df 
# 读取record_df 一行数据得到的结果
target_data_df = angle_df
attribute_info = record_df.loc[record_row, ["姓名", "穿鞋", "xsens号","实际xsens号"]]

Result_test_1 = GetEachTestResult(record_df, target_data_df, record_row, 1, road_type)
Result_test_1_info = pd.concat([attribute_info, Result_test_1])
Result_test_2 = GetEachTestResult(record_df, target_data_df, record_row, 2, road_type)
Result_test_2_info = pd.concat([attribute_info, Result_test_2])
frame = [Result_test_1_info, Result_test_2_info]
result_one_file = pd.concat(frame, axis=1)

result_one_file

,0,1
姓名,fengtianya,fengtianya
穿鞋,不穿鞋垫,不穿鞋垫
xsens号,1,1
实际xsens号,1,1
Left Hip Flexion/Extension,55.712422,59.001281
Left Hip Abduction/Adduction,0.289892,0.282185
Left Hip Internal/External Rotation,-10.837985,-3.674217
Left Knee Flexion/Extension,26.982538,32.720576
Left Knee Abduction/Adduction,1.872207,4.169345
Left Knee Internal/External Rotation,0.03474,4.126872


In [232]:
result_dataframe = pd.DataFrame(result_one_file_total).T
result_dataframe.reset_index(drop=True, inplace=True)
result_dataframe.index = result_dataframe.index + 1
result_dataframe.to_excel("temp.xlsx", index=False)

In [233]:
result_dataframe

,Left Hip Flexion/Extension,Left Hip Abduction/Adduction,Left Hip Internal/External Rotation,Left Knee Flexion/Extension,Left Knee Abduction/Adduction,Left Knee Internal/External Rotation,Left Ankle Dorsiflexion/Plantarflexion,Left Ankle Abduction/Adduction,Left Ankle Internal/External Rotation,Left Hip Flexion/Extension,...,Right Ankle Internal/External Rotation,Right Hip Flexion/Extension,Right Hip Abduction/Adduction,Right Hip Internal/External Rotation,Right Knee Flexion/Extension,Right Knee Abduction/Adduction,Right Knee Internal/External Rotation,Right Ankle Dorsiflexion/Plantarflexion,Right Ankle Abduction/Adduction,Right Ankle Internal/External Rotation
1,79.836772,0.482670,-5.414127,34.763303,0.955662,0.047499,5.016393,-0.235125,4.384963,105.756267,...,0.294874,100.869276,4.790714,4.211853,61.538518,-2.249993,-2.252759,12.876331,-6.716151,1.192796
2,27.818292,11.883130,-15.911164,10.392975,0.247760,4.403918,13.728338,12.111644,-2.193211,35.647605,...,-7.715563,61.772335,-8.059462,0.762625,25.639386,-2.215410,-1.273376,5.834369,2.830859,-5.456153
3,45.616146,0.999963,-4.106551,13.584873,-0.868712,2.261608,0.786690,1.863502,1.122368,81.890881,...,-2.982615,103.730898,4.578916,4.903123,64.924947,-2.226807,-2.252979,13.515185,-7.561203,1.416642
4,41.979076,-10.878441,-1.164955,19.303451,-2.233069,-2.469192,6.392116,3.430950,-0.199111,35.910963,...,-9.215904,52.214782,1.378666,0.771228,20.668140,-1.383064,-1.477261,2.918605,6.433391,-8.497269
